# def 
讀取

In [1]:
import pandas as pd

def read_excel(path):
    '''
    開啟excel檔案，return df

    :param path: excel path
    '''
    data = pd.ExcelFile(path)
    df = data.parse()
    print(df.columns)
    return df


In [2]:
df = read_excel(r"C:\Users\user\Desktop\Python\Martech\Python關鍵字分析\new_folder\3_ msig-mingtai(2024_08_05-2024_09_01) .xlsx")

Index(['关键词', '点击量', 'Traffic ', '变动', '28 天的体量', '流量变化', '桌面端份额', '移动端份额',
       '竞争', '意图', '平均体量', 'CPC', '零点击', '热门网址', 'SERP Features'],
      dtype='object')


刪減欄位

In [17]:
def to_three_df(df):
    '''
    刪減 col,轉換 col name, return three_df

    :param df: dataframe
    '''
    three_df_old = df[['关键词', '点击量', '热门网址']]
    three_df = three_df_old.rename(columns={'关键词':"Keyword", '点击量':"Clicks", '热门网址':"Website"})
    print(three_df.info())
    return three_df

In [ ]:
three_df = to_three_df(df)

清洗 

In [5]:
def str_to_zero(three_df):
    '''
    show non int value, return str -> 0 的 df

    :param path: three_df
    '''
    non_int = three_df["Clicks"].loc[three_df['Clicks'].apply(lambda x: not isinstance(x, int))]
    unique_value = non_int.unique()
    print(f"Unique value: {unique_value}, Non int value to zero counts: {non_int.count()}")
    three_df["Clicks"] = three_df['Clicks'].apply(lambda x: 0 if isinstance(x, str) else x)
    three_df.info()
    return three_df
    


In [6]:
df = str_to_zero(three_df)

Unique value: ['<50' '0'], Non int value to zero counts: 41
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Keyword  90 non-null     object
 1   Clicks   90 non-null     int64 
 2   Website  90 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.2+ KB


In [7]:
df.head(10)

,Keyword,Clicks,Website
0,旅平險,700,msig-mingtai.com.tw/MobileWeb/Travel/Insure/St...
1,旅遊不便險,360,msig-mingtai.com.tw/MobileWeb/Travel/Insure/In...
2,名台 機車,340,msig-mingtai.com.tw/MobileWeb/Car/Insure/Recom...
3,地震險,260,msig-mingtai.com.tw/MobileWeb/Fire/Insure
4,機車保險,230,msig-mingtai.com.tw/MobileWeb/Car/Insure/StepC...
5,明台產險旅平險,220,msig-mingtai.com.tw/MobileWeb/Travel/Insure/St...
6,汽車強制險查詢,210,msig-mingtai.com.tw/MobileWeb/Car/Insure
7,明台旅平險,210,msig-mingtai.com.tw/MobileWeb/Travel/Insure/St...
8,旅平險 國內,170,msig-mingtai.com.tw/MobileWeb/Travel/Insure
9,旅遊平安險,160,msig-mingtai.com.tw/MobileWeb/Travel/Insure/St...


# 分組

In [8]:
def group_by_kw(three_df, kw_list):
    '''
    divided dataframe by keyword(list),group by "Website" sorted by "Clicks", return contain_df and not_contain_df

    :para three_df: df
    :para kw_list: key word list
    '''
    # deal with keywork we want to use
    if isinstance(kw_list, str):
        kw_list = [kw_list]
    pattern = '|'.join(kw_list) # use '|' to concatenate keywords into regular expression

    # deal with dataframe contains keyword
    con_df = three_df.loc[three_df['Keyword'].str.contains(pattern, na=False, regex=True)] # str.contains() support re if we set the para "regex=True" 
    group_con_df = con_df.groupby("Website", as_index=False).agg({ # as_index=False: Website not as a new index
    "Clicks":"sum",
    "Keyword":lambda x: set(x)
    })  
    contain_df = group_con_df.sort_values(by="Clicks", ascending=False)
    contain_df.reset_index(drop=True, inplace=True)

    # deal with dataframe not contains keyword
    not_con_df = three_df.loc[~three_df['Keyword'].str.contains(pattern, na=False, regex=True)]
    group_not_con_df = not_con_df.groupby("Website", as_index=False).agg({ # as_index=False: Website not as a new index
    "Clicks":"sum", # groupby(): return DataFrameGroupBy object
    "Keyword":lambda x: set(x)
    })  
    not_contain_df = group_not_con_df.sort_values(by="Clicks", ascending=False)
    not_contain_df.reset_index(drop=True, inplace=True)


    return contain_df, not_contain_df


In [9]:
kw_df, no_kw_df = group_by_kw(df, kw_list=["明台","名台"])

In [10]:
kw_df.head(5)

,Website,Clicks,Keyword
0,msig-mingtai.com.tw/MobileWeb/Travel/Insure/St...,750,"{旅遊險-明台保險線上投保限定, 明台旅遊平安險, 明台產險旅平險, 明台旅平險, 明台產險..."
1,msig-mingtai.com.tw/MobileWeb/Car/Insure/Recom...,460,"{名台 機車, 明台租車險, 明台車險查詢}"
2,msig-mingtai.com.tw/MobileWeb/Travel/Insure/In...,270,"{名台 申根保險, 中華 漫遊 明台產物}"
3,msig-mingtai.com.tw/MobileWeb/Motor/Insure,160,{名台 機車險}
4,msig-mingtai.com.tw/MobileWeb/Motor/Insure/Ste...,150,"{明台機車強制險, 明台機車}"


In [11]:
no_kw_df.head(5)

,Website,Clicks,Keyword
0,msig-mingtai.com.tw/MobileWeb/Travel/Insure/St...,1040,"{出國 保險, 旅平險, 機票 不便險 國內, 旅平險 中華電信, 旅遊平安險, 旅平險 國..."
1,msig-mingtai.com.tw/MobileWeb/Car/Insure/StepC...,990,"{汽車任意險費用, 「汽車第三人責任險充電期間附加條款」, 汽車任意險, 車險試算, 汽車保..."
2,msig-mingtai.com.tw/MobileWeb/Travel/Insure/In...,920,"{旅遊不便險, 新安東京旅平險, 旅遊險, 旅遊平安險推薦, 日本 出差 保險, 嬰兒旅平險..."
3,msig-mingtai.com.tw/MobileWeb/Motor/Insure,350,"{機車 強制險 推薦, 機車強制險, 機車強制險查詢, 機車強制險線上投保, 重機保險}"
4,msig-mingtai.com.tw/MobileWeb/Car/Insure,330,"{汽車 強制險, 汽車強制險查詢, 汽車 保險, 機車強制, 汽機車強制險}"


# 儲存

In [12]:
def organic_to_excel(brand_name, kw_df, no_kw_df):
    '''
    :para brand_name: str
    :para kw_df: kw_df
    :para no_kw_df: no_kw_df
    '''
    import os
    try:
        path = r"C:\Users\user\Desktop\Python\Martech\Python關鍵字分析\organic_all_data.xlsx"
        
        if not os.path.exists(path):
            raise FileNotFoundError(f"文件不存在：{path}")
        

        save_data = pd.DataFrame()
        save_data[f"{brand_name}_有{brand_name}"] = kw_df["Keyword"].head(5)
        save_data[f"{brand_name}_沒{brand_name}"] = no_kw_df["Keyword"].head(5)
        
        try:
            # sheet 1
            data = pd.ExcelFile(path)
            existing_data = data.parse('Sheet1')
            all_data = pd.concat([existing_data, save_data], axis=1) # 合併原有數據和新的數據, axis=1沿著columns，代表處理每個row

        except FileNotFoundError:
            print(f"未找到文件")

        except ValueError:
            print("無法讀取 Sheet1") 

        with pd.ExcelWriter(path, mode='a', if_sheet_exists="overlay") as writer:
            all_data.to_excel(writer, sheet_name="Sheet1", index=False)    
            
        # new sheet
        with pd.ExcelWriter(path, mode='a', if_sheet_exists="overlay") as writer:
            kw_df.to_excel(writer, sheet_name=f'{brand_name}_有{brand_name}', index=False)
            
        # new sheet
        with pd.ExcelWriter(path, mode='a', if_sheet_exists="overlay") as writer:
            no_kw_df.to_excel(writer, sheet_name=f'{brand_name}_沒{brand_name}', index=False)
        
    except PermissionError:
        print("無法寫入文件，可能是文件已被開啟")
    except Exception as e:
        print(f"寫入 Excel 過程發生錯誤：{e}")
    
    # file not found
    except FileNotFoundError as fnf_error:
        print(f"錯誤：{fnf_error}")
    
    # OS error
    except OSError as os_error:
        print(f"OS 錯誤：{os_error}")
    
    except Exception as e:
        print(f"發生未知錯誤：{e}")
    else:
        print("文件保存成功！")

In [13]:
def paid_to_excel(brand_name, kw_df, no_kw_df):
    '''
    :para brand_name: str
    :para kw_df: kw_df
    :para no_kw_df: no_kw_df
    '''
    import os
    try:
        path = r"C:\Users\user\Desktop\Python\Martech\Python關鍵字分析\paid_all_data.xlsx"
        
        if not os.path.exists(path):
            raise FileNotFoundError(f"文件不存在：{path}")
        

        save_data = pd.DataFrame()
        save_data[f"{brand_name}_有{brand_name}"] = kw_df["Keyword"].head(5)
        save_data[f"{brand_name}_沒{brand_name}"] = no_kw_df["Keyword"].head(5)
        
        try:
            # sheet 1
            data = pd.ExcelFile(path)
            existing_data = data.parse('Sheet1')
            all_data = pd.concat([existing_data, save_data], axis=1) # 合併原有數據和新的數據, axis=1沿著columns，代表處理每個row

        except FileNotFoundError:
            print(f"未找到文件")

        except ValueError:
            print("無法讀取 Sheet1") 

        with pd.ExcelWriter(path, mode='a', if_sheet_exists="overlay") as writer:
            all_data.to_excel(writer, sheet_name="Sheet1", index=False)    
            
        # new sheet
        with pd.ExcelWriter(path, mode='a', if_sheet_exists="overlay") as writer:
            kw_df.to_excel(writer, sheet_name=f'{brand_name}paid_有{brand_name}', index=False)
            
        # new sheet
        with pd.ExcelWriter(path, mode='a', if_sheet_exists="overlay") as writer:
            no_kw_df.to_excel(writer, sheet_name=f'{brand_name}paid_沒{brand_name}', index=False)
        
    except PermissionError:
        print("無法寫入文件，可能是文件已被開啟")
    except Exception as e:
        print(f"寫入 Excel 過程發生錯誤：{e}")
    
    # file not found
    except FileNotFoundError as fnf_error:
        print(f"錯誤：{fnf_error}")
    
    # OS error
    except OSError as os_error:
        print(f"OS 錯誤：{os_error}")
    
    except Exception as e:
        print(f"發生未知錯誤：{e}")
    else:
        print("文件保存成功！")